# Agentic Strategy Construction Tool
This is to produce trade reports that can be used in the evaluation section to backtest. An output example is at the bottom of this notebook to show the agentic decision making process in more detail.

In [1]:
import copy
import pandas as pd
import requesters.company_data as cd
import agents.financial_agent as fa
import agents.committee_agent as ca
import importlib
from tqdm import tqdm
import json

from utils.s3_helper import S3Helper
from datetime import datetime

import concurrent.futures


In [19]:
importlib.reload(fa)
importlib.reload(ca)
importlib.reload(cd)

<module 'requesters.company_data' from '/project/requesters/company_data.py'>

### Collect the data needed to run the Agents
The Agents will require financial statement datasets and news datasets in order to conduct their analysis.

In [3]:
# Get the company reports
security_datasets = cd.SecurityData('tmp/fs', 'dow_quarterly_ltm_v3.json')
# get the saved news datasets
s3_helper = S3Helper('tmp/fs')
s3_helper.get_file(filename='dow_headlines.parquet', local_filename='/tmp/dow_headlines.parquet')
# Convert to pandas dataframe
news_headlines = pd.read_parquet('/tmp/dow_headlines.parquet')

### Setup the Agentic Models
Running the inference tasks across 893 date/ security combinations in 1:32h.

In [20]:
financial_agent = fa.FinancialAnalystAgent()
committee_agent = ca.CommitteeAgent()

In [21]:
def run_single(security: str, as_of_date: str) -> dict:
    """
    Function to run a single run of the Agent
    """
    company_data = security_datasets.get_security_all_data(as_of_date, security)
    # Time the run
    start_time = datetime.now()
    # Run the financial analyst agent
    financial_report = financial_agent.run(security_data=company_data, 
                                       news_data=news_headlines, 
                                       as_of_date=as_of_date)
    # Run the committee agent
    committee_report = committee_agent.run(senior_analyst_report=financial_report['senior_report'],
                                           financial_statement_analysis=financial_report['financial_report'],
                                           security_data=company_data)
    end_time = datetime.now()
    decision_dict = {
        'date': as_of_date,
        'security': security,
        'earning_decision': financial_report['final_output'].direction,
        'earning_magnitude': financial_report['final_output'].magnitude,
        'earning_confidence': financial_report['final_output'].confidence,
        'recommendation': committee_report['results'].recommendation,
        
        'responses': {'financial_analyst': financial_report,
                     'committee_report': committee_report},
        'time': str(end_time - start_time)
    }
    return decision_dict
    
    

In [22]:
# Set up the securities and security information 
dates_and_securities = security_datasets.date_security_timeseries()


### Run a single security

In [24]:
%%time
# Pick a test security
test_security = dates_and_securities[20]
# Run for a single date/ security
test_output = run_single(test_security['security'],test_security['date'])

CPU times: user 9.28 s, sys: 191 ms, total: 9.47 s
Wall time: 3min 20s


In [25]:
#dict_keys(['date', 'security', 'earning_decision', 'earning_magnitude', 'earning_confidence', 'recommendation', 'responses', 'time'])
test_output['recommendation']

'SELL'

### Run the backtest
This will run across multiple threads to speed up the process. Results are collected in the data_output list.

In [26]:
data_output = []
def backtest(dates_and_securities):
    # create the securities
    progress = tqdm(total=len(dates_and_securities), position=0, leave=True)
    # Run on 20 threads to speed up execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(run_single, single['security'], single['date']) for single in dates_and_securities]
        for f in concurrent.futures.as_completed(futures):
            progress.update(n=1)
            data_output.append(f.result())
        

In [45]:
# Run the backtest
backtest(dates_and_securities)

100%|██████████| 212/212 [29:49<00:00,  8.44s/it] 


In [46]:
len(data_output)

816

### Save the raw output to S3

In [47]:
data_analysis_final = copy.deepcopy(data_output)

In [48]:
# Some of the items are not serializable. Convert to string before JSON
for item in data_analysis_final:
    item['responses']['financial_analyst']['final_output'] = str(item['responses']['financial_analyst']['final_output'])
    item['responses']['committee_report']['results'] = str(item['responses']['committee_report']['results'])
    item['responses']['committee_report']['history'] = str(item['responses']['committee_report']['history'])
    item['time']=str(item['time'])

In [49]:
with open('/tmp/agentic_output_llama_v6.json', 'w') as f:
    json.dump(data_analysis_final, f)

In [50]:
s3_helper.add_file(local_filename='/tmp/agentic_output_llama_v6.json')

### Create the Trade Report

In [52]:
def trade_report_generation(data, trade_only=False) -> pd.DataFrame:
    """
    Function to create the trade report to pass into the Strategy Analysis tool
    """
    trade_report = []
    if trade_only:
        for item in data:
            trade_report.append({'date': item['date'], 
                                 'security': item['security'], 
                                 'decision': item['recommendation'],
                                 'confidence': item['earning_confidence']})
    
        return pd.DataFrame(data=trade_report)
    else:
        for item in data:
            trade_report.append({'date': item['date'], 
                                 'security': item['security'], 
                                 'decision': item['recommendation'],
                                 'confidence': item['earning_confidence'],
                                 'earning_decision': item['earning_decision'],
                                 'earning_magnitude': item['earning_magnitude']})
        return trade_report

In [53]:
trades = trade_report_generation(data_analysis_final)

In [54]:
with open('Results/Agentic/trades_llama_v7.json', 'w') as f:
    json.dump(trades, f)

### Example Output from agent

In [5]:
import json
from IPython.display import Markdown, display
from utils.s3_helper import S3Helper

In [6]:
s3_helper = S3Helper('tmp/fs')
s3_helper.get_file('agentic_output_llama_v6.json','/tmp/agentic_output_llama_v6.json')

In [8]:
with open('/tmp/agentic_output_llama_v6.json', 'rb') as f:
    all_data = json.load(f)

In [24]:
security = all_data[10]
response = security['responses']

security['date']

'2020-01-31'

### Financial Report

In [11]:
display(Markdown(response['financial_analyst']['financial_report']))

As an equity financial analyst, I will analyze the financial statements to predict if the company's earnings will increase in the next quarter. I will consider the current profitability, liquidity, solvency, and efficiency ratios.

**Profitability Ratios:**

1. Gross Margin Ratio = (Gross Profit / Revenue) x 100
= (2.778500e+10 / 2.608120e+11) x 100 = 10.65% (t-1)
= (3.140300e+10 / 2.715770e+11) x 100 = 11.56% (t-2)
The gross margin ratio has increased from 10.65% to 11.56% over the past year, indicating an improvement in profitability.

2. Operating Margin Ratio = (Operating Income / Revenue) x 100
= (1.375900e+10 / 2.608120e+11) x 100 = 5.27% (t-1)
= (1.729500e+10 / 2.715770e+11) x 100 = 6.37% (t-2)
The operating margin ratio has increased from 5.27% to 6.37% over the past year, indicating an improvement in profitability.

3. Net Profit Margin Ratio = (Net Income / Revenue) x 100
= (1.465000e+10 / 2.608120e+11) x 100 = 5.62% (t-1)
= (1.772000e+10 / 2.715770e+11) x 100 = 6.53% (t-2)
The net profit margin ratio has increased from 5.62% to 6.53% over the past year, indicating an improvement in profitability.

**Liquidity Ratios:**

1. Current Ratio = Current Assets / Current Liabilities
= 5.000800e+10 / 6.419500e+10 = 0.78 (t-1)
= 5.174300e+10 / 7.028700e+10 = 0.74 (t-2)
The current ratio has decreased from 0.78 to 0.74 over the past year, indicating a slight deterioration in liquidity.

2. Quick Ratio = (Current Assets - Inventory) / Current Liabilities
= (5.000800e+10 - 1.759000e+10) / 6.419500e+10 = 0.53 (t-1)
= (5.174300e+10 - 1.883500e+10) / 7.028700e+10 = 0.49 (t-2)
The quick ratio has decreased from 0.53 to 0.49 over the past year, indicating a slight deterioration in liquidity.

**Solvency Ratios:**

1. Debt-to-Equity Ratio = Total Debt / Total Equity
= 1.622520e+11 / 1.971090e+11 = 0.82 (t-1)
= 1.622640e+11 / 1.984650e+11 = 0.82 (t-2)
The debt-to-equity ratio has remained stable at 0.82 over the past year, indicating a stable solvency position.

2. Interest Coverage Ratio = EBIT / Interest Expense
= 1.375900e+10 / 8.440000e+08 = 16.31 (t-1)
= 1.729500e+10 / 8.120000e+08 = 21.29 (t-2)
The interest coverage ratio has increased from 16.31 to 21.29 over the past year, indicating an improvement in solvency.

**Efficiency Ratios:**

1. Asset Turnover Ratio = Revenue / Total Assets
= 2.608120e+11 / 3.593610e+11 = 0.73 (t-1)
= 2.715770e+11 / 3.607290e+11 = 0.75 (t-2)
The asset turnover ratio has increased from 0.73 to 0.75 over the past year, indicating an improvement in efficiency.

2. Return on Equity (ROE) = Net Income / Total Equity
= 1.465000e+10 / 1.971090e+11 = 7.43% (t-1)
= 1.772000e+10 / 1.984650e+11 = 8.92% (t-2)
The ROE has increased from 7.43% to 8.92% over the past year, indicating an improvement in profitability.

**Conclusion:**

Based on the analysis of the financial statements, I predict that the company's earnings will increase in the next quarter. The company has shown improvements in profitability, solvency, and efficiency over the past year. The gross margin ratio, operating margin ratio, and net profit margin ratio have all increased, indicating an improvement in profitability. The interest coverage ratio has also increased, indicating an improvement in solvency. The asset turnover ratio has increased, indicating an improvement in efficiency. The ROE has also increased, indicating an improvement in profitability.

**Confidence Score:** 80%

**Magnitude of Increase:** 5-10%

Note: The confidence score and magnitude of increase are subjective and based on the analysis of the financial statements.

## Financial Agent
### News Summary

In [12]:
# Headlines
response['financial_analyst']['cleaned_headlines']

'Here is the list of headlines with all references to Exxon Mobil Corp and their products removed:\n\n[\'blah ex-date estimated for feb. 10, post option expiry\', \'n.a. energy weekly\\xa0agenda: chk, eog, oas, fang, pxd, pe earnings\', \'*blah plans to divest up to $25b assets: reuters\', \'blah implied volatility surges as shares fall\', \'blah makes another guyana discovery as it raises oil estimate\', \'new york’s case that blah defrauded investors is in trouble\', \'energy stocks continue rally as oil extends gain above $58\', \'blah is said poised to kick off u.k., germany disposals\', \'lng tender: png\\xa0sells feb.-march delivery cargo at $4.70-$4.80\', \'*blah is said poised to kick off u.k., germany disposals\', \'*blah is said to start sale process soon for u.k. upstream ops\', \'blah left out of iran oil rally as grim profit warning panned\', "*png\'s kua says jan. 31 deadline set for p\'nyang deal w/ blah", \'blah starts deliveries of imo 2020 compliant fuels at new ports

In [14]:
display(Markdown(response['financial_analyst']['news_report']))

As a financial analyst, I'll start by identifying the revenue drivers for the Energy sector:

1. Oil prices: The price of crude oil is a significant driver of revenue for energy companies.
2. Natural Gas prices: The price of natural gas also impacts revenue, particularly for companies with significant natural gas production.
3. Refining margins: The difference between the cost of crude oil and the price of refined products, such as gasoline and diesel, affects refining revenue.
4. Production volumes: The amount of oil and natural gas produced by a company impacts revenue.
5. Exploration and production costs: The cost of extracting and producing oil and natural gas affects profitability.

Now, let's analyze the headlines to determine if blah will see an increase or decrease in earnings over the next quarter:

**Positive trends:**

1. Oil prices: The headlines mention oil prices extending gains above $58, which is a positive trend for energy companies.
2. New discoveries: Blah has made new discoveries in Guyana, which could lead to increased production and revenue.
3. Increased production: Blah has started oil output in Guyana and has increased its recoverable resources estimate.
4. Asset sales: Blah plans to divest up to $25 billion in assets, which could help reduce debt and improve profitability.
5. LNG sales: Blah has sold LNG cargoes at favorable prices, which could contribute to revenue growth.

**Negative trends:**

1. Decline in chemical earnings: Blah's 3Q chemical earnings declined 66% year-over-year, which could be a concern.
2. Climate-related issues: Blah has faced climate-related lawsuits and criticism, which could lead to increased costs and regulatory scrutiny.
3. Operational issues: Blah has experienced operational issues, such as flaring and emissions, which could impact production and profitability.
4. Downgrades: Blah has been downgraded by Scotiabank, which could indicate concerns about the company's performance.

**Neutral trends:**

1. Moody's affirmation: Blah's credit rating was affirmed by Moody's, but the outlook was changed to negative from stable.
2. Disposals: Blah is planning to sell assets in the UK, Germany, and Equatorial Guinea, which could be neutral or positive depending on the sale prices.

Overall, based on the headlines, I believe that blah will see a slight increase in earnings over the next quarter due to the positive trends in oil prices, new discoveries, and increased production. However, the decline in chemical earnings and climate-related issues could offset some of these gains. The company's ability to execute on its asset sales and reduce debt will be crucial in determining its future performance.

### Senior Analyst Report

In [15]:
display(Markdown(response['financial_analyst']['senior_report']))

After reviewing both reports, I agree with the narrative of earnings increase in the next quarter. The financial statements analysis suggests that the company has shown improvements in profitability, solvency, and efficiency over the past year, with increases in gross margin ratio, operating margin ratio, net profit margin ratio, interest coverage ratio, asset turnover ratio, and ROE.

The news report also supports this narrative, highlighting positive trends such as oil prices extending gains, new discoveries, increased production, asset sales, and LNG sales. While there are some negative trends, such as decline in chemical earnings and climate-related issues, I believe that the positive trends will outweigh the negative ones, leading to a slight increase in earnings over the next quarter.

I agree with the confidence score of 80% and the magnitude of increase of 5-10% as stated in the financial statements analysis. The company's ability to execute on its asset sales and reduce debt will be crucial in determining its future performance, and I believe that the company is well-positioned to achieve this.

However, I would like to note that the news report highlights some potential risks, such as climate-related issues and operational issues, which could impact the company's performance in the long run. Therefore, it's essential to continue monitoring these trends and adjust the analysis accordingly.

Overall, based on both reports, I believe that the company's earnings will increase in the next quarter, driven by positive trends in oil prices, new discoveries, and increased production.

## Committee Agent
### Buy Summary

In [17]:
display(Markdown(response['committee_report']['summary_buy']))

BUY:**Investment Committee Discussion**

After reviewing the financial statements analysis and news report, I strongly believe that the stock is undervalued and should be a BUY for the next quarter. The company's improvements in profitability, solvency, and efficiency over the past year are impressive, and I expect this trend to continue.

Using the Discounted Cash Flow (DCF) analysis, I estimate the company's intrinsic value to be around $85 per share, which is significantly higher than the current market price of $62.12. This represents a potential upside of 37%.

To calculate the intrinsic value, I used the following assumptions:

* Discount rate: 10%
* Growth rate: 5% (based on the company's historical growth rate and industry trends)
* Terminal value: 20 times the company's estimated earnings in 2025

Using the DCF model, I estimated the company's present value of future cash flows to be around $85 per share.

In addition to the DCF analysis, I also considered other valuation methods, such as the Price-to-Earnings (P/E) ratio and the Price-to-Book (P/B) ratio. The company's P/E ratio is currently around 12 times, which is lower than the industry average of 15 times. The P/B ratio is around 1.2 times, which is also lower than the industry average of 1.5 times.

Overall, I believe that the company's strong financial performance, combined with its undervalued stock price, make it an attractive investment opportunity for the next quarter.

**Recommendation:** BUY

**Target Price:** $85 per share

**Confidence Score:** 80%

**Magnitude of Increase:** 5-10%

I recommend that the investment committee consider adding this stock to our portfolio for the next quarter, with a target price of $85 per share.

### Sell Summary

In [18]:
display(Markdown(response['committee_report']['summary_sell']))

SELL:**Recommendation:** SELL

**Summary:**

While the analysis suggests that the company's earnings will increase in the next quarter, I believe that the stock is overvalued and should be sold. The historical price chart shows that the stock has been trading at a relatively high level, with a peak price of $80.80 in 2019. Despite the predicted increase in earnings, I expect the stock price to decline in the next quarter due to the following reasons:

1. **Overvaluation:** The stock's current price is higher than its historical average, indicating that it may be overvalued.
2. **Liquidity concerns:** The current ratio and quick ratio have decreased over the past year, indicating a slight deterioration in liquidity.
3. **Climate-related risks:** The news report highlights climate-related issues, which could impact the company's performance in the long run.
4. **Operational risks:** The news report also highlights operational issues, which could impact the company's performance.

**DCF Analysis:**

Using a discounted cash flow (DCF) analysis, I estimate the company's intrinsic value to be around $55 per share, which is lower than the current price. The DCF analysis assumes a 5% discount rate, a 5-year growth rate of 5%, and a terminal growth rate of 3%.

**DCF Calculation:**

1. Estimate future cash flows: $10.23 (2020), $10.72 (2021), $11.23 (2022), $11.76 (2023), and $12.31 (2024)
2. Calculate the present value of the future cash flows using the discount rate: $8.53 (2020), $8.11 (2021), $7.73 (2022), $7.39 (2023), and $7.08 (2024)
3. Calculate the terminal value: $55.15
4. Calculate the intrinsic value: $55.15

**Conclusion:**

Based on the analysis, I recommend selling the stock due to its overvaluation, liquidity concerns, climate-related risks, and operational risks. The DCF analysis suggests that the company's intrinsic value is lower than the current price, supporting the sell recommendation.

### Hold Summary

In [19]:
display(Markdown(response['committee_report']['summary_hold']))

HOLD:**Investment Committee Discussion**

After reviewing the financial statements analysis and the news report, I agree with the senior analyst's opinion that the company's earnings will increase in the next quarter. The analysis suggests that the company has shown improvements in profitability, solvency, and efficiency over the past year.

However, I would like to discuss the valuation of the stock using a Discounted Cash Flow (DCF) analysis. Based on the historical data, I estimate the company's cost of capital to be around 8%. Using a terminal growth rate of 3%, I estimate the present value of the company's future cash flows to be around $75 per share.

**DCF Analysis**

* Estimated future cash flows: $10.65 per share (based on the company's historical growth rate)
* Cost of capital: 8%
* Terminal growth rate: 3%
* Present value of future cash flows: $75 per share

**Comparison to Current Price**

The current price of the stock is around $62 per share, which is lower than the estimated present value of $75 per share. This suggests that the stock may be undervalued.

**Other Valuation Methods**

Using other valuation methods, such as the Price-to-Earnings (P/E) ratio and the Price-to-Book (P/B) ratio, we can also estimate the stock's value.

* P/E ratio: 15x (based on the company's historical P/E ratio)
* P/B ratio: 1.2 (based on the company's historical P/B ratio)

These valuation methods also suggest that the stock may be undervalued.

**Conclusion**

Based on the analysis, I believe that the stock is undervalued and has the potential to increase in value. However, I would like to advocate for a HOLD position for the next quarter, as the company's performance is subject to various risks, including climate-related issues and operational issues. I recommend monitoring the company's progress and adjusting our analysis accordingly.

**Recommendation**

HOLD the stock for the next quarter, with a target price of $75 per share.

**Confidence Score**

80%

**Magnitude of Increase**

5-10%

Note: The confidence score and magnitude of increase are subjective and based on the analysis of the financial statements and other valuation methods.

### Debate

In [21]:
response['committee_report']['history']

'[HumanMessage(content=\'Nothing\', additional_kwargs={}, response_metadata={}, id=\'0cc4e6d1-8d24-4349-8c9d-b3af34caa940\'), HumanMessage(content="BUY:**Investment Committee Discussion**\\n\\nAfter reviewing the financial statements analysis and news report, I strongly believe that the stock is undervalued and should be a BUY for the next quarter. The company\'s improvements in profitability, solvency, and efficiency over the past year are impressive, and I expect this trend to continue.\\n\\nUsing the Discounted Cash Flow (DCF) analysis, I estimate the company\'s intrinsic value to be around $85 per share, which is significantly higher than the current market price of $62.12. This represents a potential upside of 37%.\\n\\nTo calculate the intrinsic value, I used the following assumptions:\\n\\n* Discount rate: 10%\\n* Growth rate: 5% (based on the company\'s historical growth rate and industry trends)\\n* Terminal value: 20 times the company\'s estimated earnings in 2025\\n\\nUsing 

### Final result from the Committee Agent

In [22]:
response['committee_report']['results']

"recommendation='BUY' reason='The majority voting pattern is 2 BUY and 1 HOLD. The senior analyst and most committee members support a BUY recommendation based on:\\n1. Positive financial performance with improvements in profitability, solvency, and efficiency\\n2. Positive trends in oil prices, new discoveries, and increased production\\n3. Discounted Cash Flow (DCF) analysis suggesting the stock is undervalued\\n4. Estimated intrinsic value ranging from $75-$85 per share (compared to current price of $62.12)\\n5. Potential upside of 20-37%\\n\\nWhile there are some risks such as climate-related and operational issues, the overall sentiment is positive with a high confidence score of 80% and expected earnings increase of 5-10%.' confidence='High'"

### Unused code

In [1]:
# from utils.s3_helper import S3Helper

# s3_helper = S3Helper('tmp/fs')
# s3_helper.get_file(filename='agentic_output_llama_v3_1.json',local_filename='/tmp/agentic_output_llama_v3_1.json')

In [2]:
# import json
# with open('/tmp/agentic_output_llama_v3_1.json', 'rb') as f:
#     data_output = json.load(f)

In [1]:
# print(data_output[1])

In [2]:

s3_helper.get_file('agentic_output_llama_v6.json','/tmp/agentic_output_llama_v6.json')

NameError: name 's3_helper' is not defined